In [1]:
import numpy as np 
from sklearn.model_selection import train_test_split
import utils_config
import tensorflow as tf

from matplotlib import pyplot as plt

In [2]:
np.random.seed(2021)

## Load Data

In [3]:
np_grid_data = np.load(r'_np_griddata.npy')
np_init_path = np.load(r'_np_initpath.npy')
np_opt_path = np.load(r'_np_optpath.npy')

In [4]:
np_grid_data = np_grid_data.astype('float32')/127.0
np_grid_data = np.reshape(np_grid_data,(np_grid_data.shape[0],np_grid_data.shape[1],np_grid_data.shape[1],1))
print(np_grid_data.max())
print("Shape of grid data:",np.shape(np_grid_data))
print("Shape of init path:",np.shape(np_init_path))
print("Shape of opt path:",np.shape(np_opt_path))

#plt.matshow(np_grid_data[0])

1.0
Shape of grid data: (227, 1536, 1536, 1)
Shape of init path: (227, 25, 2)
Shape of opt path: (227, 25, 2)


## Check Data balance 

In [5]:
# Check Data Balance
count=0

for i in range(len(np_opt_path)):
    if np.array_equiv(np_opt_path[i],np_init_path[i]):
        count +=1

print("Paths that are similar: {}, which is {} of total data".format(count,(count/len(np_init_path))))
print("Paths that are dissimalar: {},which is {} of total data".format((len(np_init_path)-count),((len(np_init_path)-count)/len(np_init_path))))

Paths that are similar: 207, which is 0.9118942731277533 of total data
Paths that are dissimalar: 20,which is 0.0881057268722467 of total data


## Build Data set

In [6]:
# Split dataset
train_grid,test_grid,train_init_path,test_init_path,train_opt_path,test_opt_path = train_test_split(np_grid_data,np_init_path,np_opt_path,random_state=np.random.randint(1),test_size=0.25)

In [7]:
'''print sizes and shapes of testing and training data'''

print("Shape of training grid data:",np.shape(train_grid))
print("Shape of training init path:",np.shape(train_init_path))
print("Shape of training opt path:",np.shape(train_opt_path))
print("-----------------------------------------------------")
print("Shape of testing grid data:",np.shape(test_grid))
print("Shape of testing init path:",np.shape(test_init_path))
print("Shape of testing opt path:",np.shape(test_opt_path))

Shape of training grid data: (170, 1536, 1536, 1)
Shape of training init path: (170, 25, 2)
Shape of training opt path: (170, 25, 2)
-----------------------------------------------------
Shape of testing grid data: (57, 1536, 1536, 1)
Shape of testing init path: (57, 25, 2)
Shape of testing opt path: (57, 25, 2)


In [8]:
def parse_func(input_data,optimized_path):
    
    '''    
    image_string = tf.io.read_file(filename)
    
    image_decoded = tf.io.decode_jpeg(image_string, channels=3)
   
    image = tf.image.resize(image_decoded, size=(util_constants.IMG_SIZE_VGG[0],util_constants.IMG_SIZE_VGG[1]))
    
    # add random noise to image
    noise = tf.random.normal(image.shape,0,3)
    
    image = image + noise
    
    image = preprocess_input(image)
    '''
    grid_data = input_data[0]
    
    init_path = input_data[1]
    
    grid_data = tf.cast(grid_data,tf.float16)
    grid_data = grid_data/tf.norm(grid_data)
    
    modified_input = (grid_data,init_path)
    return modified_input,optimized_path


AUTOTUNE = tf.data.experimental.AUTOTUNE

def build_dataset(grid,init_path,opt_path,data_set_type):
    
    ds = tf.data.Dataset.from_tensor_slices(((grid,init_path),opt_path))
    
    if data_set_type=='train':
        p_var = data_set_type
        print("####### Buildling {} data set..... ########".format(p_var))
        ds = ds.shuffle(utils_config.N_SHUFFLE_BUFFER)
        ds = ds.cache()
        ds = ds.map(parse_func, num_parallel_calls = AUTOTUNE)
      
    if not data_set_type=='train':
        print("Buildling {} data set".format(data_set_type))
        ds = ds.cache()
        #ds = ds.map(parse_func, AUTOTUNE)
    
    ds = ds.batch(utils_config.H_BATCH_SIZE,drop_remainder=True).prefetch(AUTOTUNE)
    
    print(ds.element_spec)
    
    return ds



In [9]:
ds_train = build_dataset(train_grid,train_init_path,train_opt_path,'train')
ds_test = build_dataset(test_grid,test_init_path,test_opt_path,'test')

####### Buildling train data set..... ########
((TensorSpec(shape=(16, 1536, 1536, 1), dtype=tf.float16, name=None), TensorSpec(shape=(16, 25, 2), dtype=tf.float64, name=None)), TensorSpec(shape=(16, 25, 2), dtype=tf.float64, name=None))
Buildling test data set
((TensorSpec(shape=(16, 1536, 1536, 1), dtype=tf.float32, name=None), TensorSpec(shape=(16, 25, 2), dtype=tf.float64, name=None)), TensorSpec(shape=(16, 25, 2), dtype=tf.float64, name=None))


In [10]:
#check the tensorflow dataset   

#next(iter(ds_train))
for (input_data,output_data) in ds_train.take(1):
    np_one_grid = input_data[0].numpy()
    np_one_grid = np_one_grid[0]
    print((np_one_grid.max()))
    
    #(plt.matshow(input_data[0].numpy()[0]))
#print(plt.matshow(np.reshape(np_one_grid,(1536,1536))))

0.006798


In [11]:
from tensorflow.keras import layers
from tensorflow import keras
def nn():

    # define two sets of inputs
    input_A = layers.Input(shape=(1536,1536,1))
    input_B = layers.Input(shape=(25,2))

    reshape_input_B = layers.Reshape((50,))(input_B)

    #CNN
    x_A = layers.Conv2D(32,kernel_size=3,strides=2,activation='relu')(input_A)
    x_A = layers.BatchNormalization()(x_A)
    x_A = layers.AvgPool2D(pool_size=(4,4))(x_A)

    x_A = layers.Conv2D(64,kernel_size=3,strides=2,activation='relu')(x_A)
    x_A = layers.BatchNormalization()(x_A)
    x_A = layers.AvgPool2D(pool_size=(4,4))(x_A)

    x_A = layers.Flatten()(x_A)
    
    #concatenate feature
    concat_feat = layers.concatenate([x_A, reshape_input_B])

    #FC
    output = layers.Dense(50, activation='linear')(concat_feat)
    output = layers.Dense(50, activation='linear')(output)
    output = layers.Reshape((25,2))(output)
    nn_fun = keras.models.Model(inputs = [input_A, input_B], outputs= output)
    
    return nn_fun
    

In [12]:
pp_model = nn()
pp_model.compile(optimizer='adam',loss= 'mse', metrics= 'accuracy')

In [13]:
pp_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1536, 1536,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 767, 767, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 767, 767, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
average_pooling2d (AveragePooli (None, 191, 191, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
pp_model.fit(ds_train,epochs=5)

Epoch 1/5
10/10 [==============================] - 87s 8s/step - loss: 507.1895 - accuracy: 0.6566
Epoch 2/5
10/10 [==============================] - 91s 9s/step - loss: 194.5522 - accuracy: 0.9467
Epoch 3/5
10/10 [==============================] - 107s 11s/step - loss: 103.3802 - accuracy: 0.9568
Epoch 4/5
10/10 [==============================] - 106s 10s/step - loss: 74.8188 - accuracy: 0.9371
Epoch 5/5
10/10 [==============================] - 84s 8s/step - loss: 51.2319 - accuracy: 0.9285


In [11]:
list_number = [0,1,2,3,4,5,6,7,98,5]

In [12]:
list_number[-1]

5